In [ ]:
from pathlib import Path
import os

In [ ]:
fp = os.path.join(Path().absolute(), "inputs", "input18.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input18_test.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [ ]:
data

# Part 1

In [ ]:
data_processed = []
for line in data:
    direction, num, colour = line.split()
    num = int(num)
    colour = colour[1:-1]
    data_processed.append((direction, num, colour))

In [ ]:
data_processed

In [ ]:
first_dir = data_processed[0][0]
assert first_dir == "R"
current_x = 0
current_y = 0
current = (current_x, current_y)

boundary = [current]

for direction, num, _ in data_processed:

    if direction == "R":
        new = [(current_x, current_y + n) for n in range(1, num + 1)]
    if direction == "L":
        new = [(current_x, current_y - n) for n in range(1, num + 1)]
    if direction == "D":
        new = [(current_x + n, current_y) for n in range(1, num + 1)]
    if direction == "U":
        new = [(current_x - n, current_y) for n in range(1, num + 1)]

    boundary += new
    current = new[-1]
    current_x, current_y = current

assert boundary[0] == boundary[-1]
boundary = boundary[:-1]

In [ ]:
len(boundary)

In [ ]:
x_max = max(x for x, y in boundary)
x_min = min(x for x, y in boundary)

y_max = max(y for x, y in boundary)
y_min = min(y for x, y in boundary)

num_rows = x_max - x_min + 1
num_cols = y_max - y_min + 1
num_cells = num_rows * num_cols
print(num_rows, num_cols, num_cells)

grid_corners = [(x_min, y_min), (x_min, y_max), (x_max, y_min), (x_max, y_max)]

In [ ]:
def get_neighbours(point):
    
    x, y = point
    res = []
    if x > x_min:
        res.append((x - 1, y))
    if x < x_max:
        res.append((x + 1, y))
    if y > y_min:
        res.append((x, y - 1))
    if y < y_max:
        res.append((x, y + 1))

    return res

In [ ]:
get_neighbours((1, 3))

In [ ]:
boundary[:10]

In [ ]:
def no_blocker(pos1_x, pos1_y, pos2_x, pos2_y):
    # check if no valid blocking connection
    return not ((pos1_x, pos1_y) in boundary and (pos2_x, pos2_y) in boundary)

In [ ]:
grid_corners

In [ ]:
top_left_grid_corner = (x_min, y_min)
top_right_grid_corner = (x_min, y_max)
bottom_left_grid_corner = (x_max, y_min)
bottom_right_grid_corner = (x_max, y_max)

In [ ]:
outside_intersections_expanded = []

outside_grid_corners = [grid_corner for grid_corner in grid_corners if grid_corner not in boundary]

# intersection anchoring relative to cells
if len(outside_grid_corners) == 0:
    raise ValueError("All inside")
else:
    outside_grid_corner = outside_grid_corners[0]
    if outside_grid_corner == top_left_grid_corner:
        start = (outside_grid_corner[0], outside_grid_corner[1])
    elif outside_grid_corner == top_right_grid_corner:
        start = (outside_grid_corner[0], outside_grid_corner[1] + 1)
    elif outside_grid_corner == bottom_left_grid_corner:
        start = (outside_grid_corner[0] + 1, outside_grid_corner[1])
    elif outside_grid_corner == bottom_right_grid_corner:
        start = (outside_grid_corner[0] + 1, outside_grid_corner[1] + 1)
    else:
        raise ValueError
    
outside_to_expand = [start]


max_num_iter = 200_000
num_iter = 0

while len(outside_to_expand) > 0 and num_iter < max_num_iter:
    # print(f"Expanded: {outside_intersections_expanded}, to expand: {outside_to_expand}")
    print(f"Expanded: {len(outside_intersections_expanded)}, to expand: {len(outside_to_expand)}")

    intersection_to_expand = outside_to_expand.pop()
    x, y = intersection_to_expand

    # up
    if x > x_min:
        new = (x - 1, y)
        if new not in outside_to_expand and new not in outside_intersections_expanded:
            if y == y_min or y == y_max + 1:
                # border
                outside_to_expand.append(new)
            else:
                pos1_x, pos1_y = x - 1, y - 1
                pos2_x, pos2_y = x - 1, y
                if no_blocker(pos1_x, pos1_y, pos2_x, pos2_y):
                    outside_to_expand.append(new)
    
    # down
    if x <= x_max:
        new = (x + 1, y)
        if new not in outside_to_expand and new not in outside_intersections_expanded:
            if y == y_min or y == y_max + 1:
                outside_to_expand.append(new)
            else:
                pos1_x, pos1_y = x, y - 1
                pos2_x, pos2_y = x, y
                if no_blocker(pos1_x, pos1_y, pos2_x, pos2_y):
                    outside_to_expand.append(new)

    # left
    if y > y_min:
        new = (x, y - 1)
        if new not in outside_to_expand and new not in outside_intersections_expanded:
            if x == x_min or x == x_max + 1:
                outside_to_expand.append(new)
            else:
                pos1_x, pos1_y = x - 1, y - 1
                pos2_x, pos2_y = x, y - 1
                if no_blocker(pos1_x, pos1_y, pos2_x, pos2_y):
                    outside_to_expand.append(new)

    # right
    if y <= y_max:
        new = (x, y + 1)
        if new not in outside_to_expand and new not in outside_intersections_expanded:
            if x == x_min or x == x_max + 1:
                outside_to_expand.append(new)
            else:
                pos1_x, pos1_y = x - 1, y
                pos2_x, pos2_y = x, y
                if no_blocker(pos1_x, pos1_y, pos2_x, pos2_y):
                    outside_to_expand.append(new)

    outside_intersections_expanded.append(intersection_to_expand)
    num_iter += 1

print("done", num_iter)

In [ ]:
num_cells_outside = 0
num_cells_inside = 0

boundary_set = set(boundary)
outside_intersections_expanded_set = set(outside_intersections_expanded)

for i in range(x_min, x_max):
    # print(i)
    for j in range(y_min, y_max):
        if (i, j) not in boundary_set:
            if any(x in outside_intersections_expanded_set for x in [(i, j), (i + 1, j), (i, j + 1), (i + 1, j + 1)]):
                # cell is outside
                num_cells_outside += 1
            else:
                num_cells_inside += 1

print(num_cells_outside)
print(num_cells_inside)

In [ ]:
len(boundary) + num_cells_inside

# Part 2

In [ ]:
data_corrected = []

for _, _, colour in data_processed:
    num_hex = colour[1:-1]
    num = int(num_hex, 16) # convert hexadecimal to decimal int

    dir = colour[-1]
    if dir == "0":
        dir = "R"
    if dir == "1":
        dir = "D"
    if dir == "2":
        dir = "L"
    if dir == "3":
        dir = "U"

    data_corrected.append((dir, num, None))

In [ ]:
data_corrected

In [ ]:
clockwise = {"U": "R", "R": "D", "D": "L", "L": "U", "U": "R"}

In [ ]:
first_dir = data_corrected[0][0]

# vertices are at intersections (not cells)
current_x = 0
current_y = 0
current_vertex = (current_x, current_y)

vertices = [current_vertex]

num_boundary_cells = 0
last_dir = data_processed[-1][0]

num_clockwise_turns = 0
num_counterclockwise_turns = 0

for direction, num, _ in data_corrected:

    if direction == "R":
        new_vertex = (current_x, current_y + num)
    elif direction == "L":
        new_vertex = (current_x, current_y - num)
    elif direction == "D":
        new_vertex = (current_x + num, current_y)
    elif direction == "U":
        new_vertex = (current_x - num, current_y)

    vertices.append(new_vertex)
    current_vertex = new_vertex
    current_x, current_y = current_vertex

    num_boundary_cells += num

    if clockwise[last_dir] == direction:
        num_clockwise_turns += 1
    else:
        num_counterclockwise_turns += 1

    last_dir = direction

assert vertices[0] == vertices[-1]
vertices = vertices[:-1]
num_vertices = len(vertices)

In [ ]:
vertices

In [ ]:
print(num_counterclockwise_turns)
print(num_clockwise_turns)
print(num_vertices)
print(num_boundary_cells)

In [ ]:
vertices = [vertices[-1]] + vertices + [vertices[0]]

Use Trapezoid formula here
https://en.wikipedia.org/wiki/Shoelace_formula

In [ ]:
area = 0
for i in range(1, num_vertices + 1):
    area += (vertices[i][1] + vertices[i + 1][1]) * (vertices[i][0] - vertices[i + 1][0])

area = area / 2

if area < 0:
    # clockwise
    area = abs(area)
    area = area + 3/4 * num_clockwise_turns + 1/4 * num_counterclockwise_turns + (num_boundary_cells - num_vertices) / 2
else:
    # counterclockwise
    area = area + 1/4 * num_clockwise_turns + 3/4 * num_counterclockwise_turns + (num_boundary_cells - num_vertices) / 2

print(area)